Data comes from [The Extrasolar Planet Encyclopedia](http://exoplanet.eu/). Thanks to Ilya Marchenko for sharing this dataset on [Kaggle](https://www.kaggle.com/ilyamarchenko/full-exoplanet-catalog?select=exoplanet_confirm_and_candidates.csv).

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
exo = pd.read_csv('/content/drive/My Drive/exoplanets.csv')
exo.head()

,# name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,semi_major_axis_error_min,semi_major_axis_error_max,eccentricity,eccentricity_error_min,eccentricity_error_max,inclination,inclination_error_min,inclination_error_max,angular_distance,discovered,updated,omega,omega_error_min,omega_error_max,tperi,tperi_error_min,tperi_error_max,tconj,tconj_error_min,tconj_error_max,tzero_tr,tzero_tr_error_min,tzero_tr_error_max,tzero_tr_sec,tzero_tr_sec_error_min,...,geometric_albedo,geometric_albedo_error_min,geometric_albedo_error_max,log_g,publication,detection_type,mass_detection_type,radius_detection_type,alternate_names,molecules,star_name,ra,dec,mag_v,mag_i,mag_j,mag_h,mag_k,star_distance,star_distance_error_min,star_distance_error_max,star_metallicity,star_metallicity_error_min,star_metallicity_error_max,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,11 Com b,Confirmed,NaN,NaN,NaN,19.40,1.50,1.50,NaN,NaN,NaN,326.03,0.32,0.32,1.29,0.05,0.05,0.231,0.005,0.005,NaN,NaN,NaN,0.011664,2008.0,2015-08-21,94.80,1.50,1.50,2452899.60,1.60,1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Published in a refereed paper,Radial Velocity,NaN,NaN,NaN,NaN,11 Com,185.179167,17.792778,4.74,NaN,NaN,NaN,NaN,110.6,10.5,10.5,-0.350,0.09,0.09,2.7000,0.300,0.300,19.00,2.00,2.00,G8 III,NaN,NaN,NaN,4742.0,100.0,100.0,NaN,NaN,NaN
1,11 Oph b,Confirmed,21.00,3.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,730000.00,365000.00,365000.00,243.00,55.00,55.00,NaN,NaN,NaN,NaN,NaN,NaN,1.675862,2007.0,2018-06-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Published in a refereed paper,Imaging,NaN,NaN,Oph 1622-2405 b,NaN,11 Oph,245.604167,-24.087222,NaN,NaN,NaN,NaN,14.03,145.0,20.0,20.0,NaN,NaN,NaN,0.0162,0.005,0.005,NaN,NaN,NaN,M9,0.011,0.002,0.002,2375.0,175.0,175.0,NaN,NaN,"Oph 1622-2405, Oph 11A"
2,11 UMi b,Confirmed,NaN,NaN,NaN,10.50,2.47,2.47,NaN,NaN,NaN,516.22,3.25,3.25,1.54,0.07,0.07,0.080,0.030,0.030,NaN,NaN,NaN,0.012887,2009.0,2009-08-12,117.63,21.06,21.06,2452861.05,2.06,2.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Published in a refereed paper,Radial Velocity,NaN,NaN,NaN,NaN,11 UMi,229.275000,71.823889,5.02,NaN,NaN,NaN,NaN,119.5,6.9,6.9,0.040,0.04,0.04,1.8000,0.250,0.250,24.08,1.84,1.84,K4III,1.560,0.540,0.540,4340.0,70.0,70.0,NaN,NaN,NaN
3,11 Uma b,Unconfirmed,3.72,0.82,0.82,NaN,NaN,NaN,NaN,NaN,NaN,651.90,7.10,7.10,1.56,0.07,0.07,0.700,0.070,0.070,NaN,NaN,NaN,0.049367,2008.0,2010-11-25,6.24,8.90,8.90,53469.30,5.00,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Published in a refereed paper,Default,NaN,NaN,NaN,NaN,11 Uma,137.095833,66.873333,5.16,NaN,NaN,NaN,NaN,31.6,2.9,2.9,-0.245,0.04,0.04,1.8100,0.195,0.195,36.30,4.20,4.20,K5III,NaN,NaN,NaN,4090.0,70.0,70.0,NaN,NaN,NaN
4,14 And b,Confirmed,NaN,NaN,NaN,5.33,0.57,0.57,NaN,NaN,NaN,185.84,0.23,0.23,0.83,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.010864,2008.0,2014-08-27,NaN,NaN,NaN,2452861.40,1.50,1.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Published in a refereed paper,Radial Velocity,NaN,NaN,NaN,NaN,14 And,352.820833,39.236111,5.22,4.1,3.02,2.61,2.33,76.4,4.1,4.1,-0.240,NaN,NaN,2.2000,0.200,0.200,11.00,1.00,1.00,K0III,NaN,NaN,NaN,4813.0,20.0,20.0,NaN,NaN,NaN


In [ ]:
exo = exo.loc[:, ['radius', 'mass', 'planet_status', 'orbital_period', 'star_distance']] 

In [ ]:
print("\nUnique values\n",exo.nunique())
print("\nNull values\n\n", exo.isna().sum())


Unique values
 radius            1572
mass              1057
planet_status        5
orbital_period    7050
star_distance     2532
dtype: int64

Null values

 radius            1504
mass              5815
planet_status        0
orbital_period     335
star_distance     2665
dtype: int64


## Create dummy example data

For all techniques we'll first demonstrate them on the simple DataFrame created below, then on the more realistic CSV file.

In [ ]:
import numpy as np

alien_species = {"alien_height":[80, 63, 70, 93, np.nan], "alien_age":[12, np.nan, 87, 415, 892], "home_planet":["Mars", "Jupiter", "Europa", "Mars", "Europa"]}

alien_df = pd.DataFrame(alien_species)
alien_df.head()

,alien_height,alien_age,home_planet
0,80.0,12.0,Mars
1,63.0,NaN,Jupiter
2,70.0,87.0,Europa
3,93.0,415.0,Mars
4,NaN,892.0,Europa


## Imputation
First the simple DataFrame.

### `alien_df` Example

In [ ]:
from sklearn.impute import SimpleImputer
features = alien_df.loc[:, ["alien_height", "alien_age"]]
print(features.head(), "\n")
imp = SimpleImputer()
imp.fit(features)
imputed = imp.transform(features)

# the rest of this code block reformats the data to print it in an educative way. don't sweat it!
# scikit learn often strips the column headers (it's due to converting arrays to numpy for math), so add them back like so:
imputed_alien_df = pd.DataFrame(imputed,columns=features.columns)
print(imputed_alien_df.head())
# adding back the categorical data
imputed_alien_df["home_planet"] = alien_df["home_planet"]

   alien_height  alien_age
0          80.0       12.0
1          63.0        NaN
2          70.0       87.0
3          93.0      415.0
4           NaN      892.0 

   alien_height  alien_age
0          80.0       12.0
1          63.0      351.5
2          70.0       87.0
3          93.0      415.0
4          76.5      892.0


Now let's perform imputation on the exoplanets dataset.

### Exoplanets Example

In [ ]:
exo_numbers = exo.loc[:, ['radius', 'mass', 'orbital_period', 'star_distance']]
print(exo.head())
print("\nNull values\n\n", exo.isna().sum(), "\n")
imp = SimpleImputer()
imp.fit(exo_numbers)
imputed = imp.transform(exo_numbers)
imputed_exo_df = pd.DataFrame(imputed,columns=exo_numbers.columns)

# reformatting the imputed data below
imputed_exo_df = pd.DataFrame(imputed,columns=exo_numbers.columns)
# adding back in the categorical data
imputed_exo_df["planet_status"] = exo["planet_status"]
print(imputed_exo_df.head())
print("\nNull values\n\n", imputed_exo_df.isna().sum())

   radius   mass planet_status  orbital_period  star_distance
0     NaN    NaN     Confirmed          326.03          110.6
1     NaN  21.00     Confirmed       730000.00          145.0
2     NaN    NaN     Confirmed          516.22          119.5
3     NaN   3.72   Unconfirmed          651.90           31.6
4     NaN    NaN     Confirmed          185.84           76.4

Null values

 radius            1504
mass              5815
planet_status        0
orbital_period     335
star_distance     2665
dtype: int64 

     radius       mass  orbital_period  star_distance planet_status
0  5.946769   6.872148          326.03          110.6     Confirmed
1  5.946769  21.000000       730000.00          145.0     Confirmed
2  5.946769   6.872148          516.22          119.5     Confirmed
3  5.946769   3.720000          651.90           31.6   Unconfirmed
4  5.946769   6.872148          185.84           76.4     Confirmed

Null values

 radius            0
mass              0
orbital_period    0


## One-Hot Encoding
First the `alien_df` data.

### `alien_df` Example

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
# passthrough means keep the columns that aren't encoded
# now connect it to the imputed dataframe
encoded_alien_array = np.array(ct.fit_transform(imputed_alien_df))
print(imputed_alien_df.head(), "\n")

# one hot encoding returns an array which our ML algo assumes is a NumPy array
# in the next two lines we reformat this data to be more readable
enc_alien_df = pd.DataFrame(encoded_alien_array, columns=["europa", "jupiter", "mars", "alien_height","alien_age"])
enc_alien_df = enc_alien_df.loc[:,["alien_height","alien_age","mars","jupiter","europa"]]
print(enc_alien_df.head())

Now the exoplanet dataset.

### Exoplanets Example

In [ ]:
print(imputed_exo_df.loc[:, "planet_status"].unique())
imputed_exo_df.head()

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [4])], remainder='passthrough')
enc_exo = np.array(ct.fit_transform(imputed_exo_df))

In [ ]:
# in this code block, we'll simply make the output more readable
enc_exo_df = pd.DataFrame(enc_exo)
# we can use the 'set_option` method to show all the columns here
# (the remaining columns wrap to the next line)
pd.set_option('display.max_columns', 12)
# same as last time, the next two lines just make our encoding readable
enc_exo_df.columns = ["candidate","confirmed","controversial","retracted","unconfirmed", "radius","mass", "orbital_period","star_distance"]
enc_exo_df = enc_exo_df.loc[:,["radius","mass","orbital_period","star_distance","candidate","confirmed","controversial","retracted","unconfirmed"]]

print(imputed_exo_df.head(), "\n")
print(enc_exo_df.head())

## Further Practice
If you want to sharpen your skills, try feature scaling the exoplanet data below. Then you'd have a wholly preprocessed dataset!

(Since the one-hot encoded data occurs within the normal range of standard deviation, you don't need to worry about scaling it.)

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# remember how this goes?